In [ ]:
%matplotlib notebook

import astra
import tomopy

import numpy as np
import scipy.fftpack as pfft
import matplotlib.pyplot as plt
import hyperspy.api as hs

from skimage import data_dir
from skimage.io import imread, imsave
from skimage.transform import radon, rescale, rotate, iradon,iradon_sart

# Phantom

### Skimage

Generating a phantom and the angles

In [ ]:
image = imread(data_dir + "/phantom.png", as_gray=True)

Setting the number of projections for the phantom simulations

In [ ]:
nb_proj = 45  # 180 images -> angular increment: 1°

Generating projections with radon and reproject them with iradon

In [ ]:
angles = np.linspace(0., 180., nb_proj, endpoint=False)
print(angles.shape)

sinogram = radon(image, theta=angles, circle=True)

print(sinogram.shape)
reconstruction_fbp = iradon(sinogram, theta=angles, circle=True)
reconstruction_sart = iradon_sart(sinogram, theta=angles, relaxation=0.15)

In [ ]:
sinogram.shape

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(8, 2.25))
ax1.set_title("Original")
ax1.imshow(image, cmap=plt.cm.Greys_r)
ax2.set_title("Radon transform\n(Sinogram)")
ax2.set_xlabel("Projection angle (deg)")
ax2.set_ylabel("Projection position (pixels)")
ax2.imshow(sinogram, cmap=plt.cm.Greys_r,
           extent=(0, 180, 0, sinogram.shape[0]), aspect='auto')
ax2.set_xlabel("Projection angle (deg)")
ax2.set_ylabel("Projection position (pixels)")
ax3.imshow(reconstruction_fbp, cmap=plt.cm.Greys_r)
ax3.set_title("Reconstruction FBP")
ax4.imshow(reconstruction_sart, cmap=plt.cm.Greys_r)
ax4.set_title("Reconstruction SART")
plt.show()
fig.tight_layout()
plt.show()

### Astra-toolbox

Just two remarks : the sinograms are tansposed in comparison with skimage and the angles are in radians.

Astra is a wrapping around C code and CUDA code to run on GPU. It also needs some memory management (hence the initial geometry and volume declarations and final deletions to free the memory).

In [ ]:
angles_rad = (np.pi/180)*angles
vol_geom = astra.create_vol_geom(image.shape[0],image.shape[1])
proj_geom = astra.create_proj_geom('parallel', 1.0, image.shape[0], angles_rad)

# For CPU-based algorithms, a "projector" object specifies the projection
# model used. In this case, we use the "strip" model.
proj_id = astra.create_projector('strip', proj_geom, vol_geom)

# Create a sinogram from a phantom
sinogram_id, sinogram = astra.create_sino(image, proj_id)

# Create a data object for the reconstruction
#sinogram_id =astra.data2d.create('-sino',proj_geom,sinogram.T)
rec_id = astra.data2d.create('-vol', vol_geom)

# Set up the parameters for a reconstruction algorithm using the CPU
# The main difference with the configuration of a GPU algorithm is the
# extra ProjectorId setting.
cfg = astra.astra_dict('SIRT')
cfg['ReconstructionDataId'] = rec_id
cfg['ProjectionDataId'] = sinogram_id
cfg['ProjectorId'] = proj_id
cfg['option']= {'MinConstraint' : 0,
                'MaxConstraint' : 255}

# Available algorithms:
# ART, SART, SIRT, CGLS, FBP

# Create the algorithm object from the configuration structure
alg_id = astra.algorithm.create(cfg)

# Run 20 iterations of the algorithm
# This will have a runtime in the order of 10 seconds.
astra.algorithm.run(alg_id,40)

# Get the result
rec_astra = astra.data2d.get(rec_id)

# Clean up.
astra.algorithm.delete(alg_id)
astra.data2d.delete(rec_id)
astra.data2d.delete(sinogram_id)
astra.projector.delete(proj_id)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 2.25))
ax1.set_title("Original")
ax1.imshow(image, cmap=plt.cm.Greys_r)
ax2.set_title("Radon transform\n(Sinogram)")
ax2.set_xlabel("Projection angle (deg)")
ax2.set_ylabel("Projection position (pixels)")
ax2.imshow(sinogram, cmap=plt.cm.Greys_r,
           extent=(0, 180, 0, sinogram.shape[0]), aspect='auto')
ax2.set_xlabel("Projection angle (deg)")
ax2.set_ylabel("Projection position (pixels)")
ax3.imshow(rec_astra, cmap=plt.cm.Greys_r)
ax3.set_title("Reconstruction SIRT")
plt.show()
fig.tight_layout()
plt.show()

### Tomopy
Angles in radians too. 
Can be used to call the Astra methods.



In [ ]:
phantom = [image]
theta = tomopy.angles(nb_proj)
sinogram = tomopy.project(phantom,theta)
reconstruction_TV = tomopy.recon(sinogram,theta,algorithm = 'tv',num_iter=200,reg_par=0.25, num_gridx=image.shape[0], num_gridy=image.shape[1])

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 2.25))
ax1.set_title("Original")
ax1.imshow(phantom[0], cmap=plt.cm.Greys_r)
ax2.set_title("Radon transform\n(Sinogram)")
ax2.set_xlabel("Projection angle (deg)")
ax2.set_ylabel("Projection position (pixels)")
ax2.imshow(sinogram[:,0], cmap=plt.cm.Greys_r,
           extent=(0, 180, 0, sinogram.shape[0]), aspect='auto')
ax2.set_xlabel("Projection angle (deg)")
ax2.set_ylabel("Projection position (pixels)")
ax3.imshow(reconstruction_TV[0], cmap=plt.cm.Greys_r)
ax3.set_title("Reconstruction TV")
plt.show()
fig.tight_layout()
plt.show()

## Summary

In [ ]:
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(8, 2.25))
ax1.set_title("Original")
ax1.imshow(image, cmap=plt.cm.Greys_r)
ax2.set_title("FBP")
ax2.imshow(reconstruction_fbp, cmap=plt.cm.Greys_r)
ax3.set_title("SART")
ax3.imshow(reconstruction_sart, cmap=plt.cm.Greys_r)
ax4.imshow(rec_astra, cmap=plt.cm.Greys_r)
ax4.set_title("SIRT")
ax5.imshow(reconstruction_TV[0], cmap=plt.cm.Greys_r)
ax5.set_title("TV")
plt.show()
fig.tight_layout()
plt.show()

## Real data now 

In [ ]:
Stack = imread('ali_crop_tilt-2.tif')

theta = np.linspace(0., 180., Stack.shape[0], endpoint=False)

print(Stack.shape)
print(theta.shape)

hs_stack = hs.signals.Signal2D(Stack)
hs_stack.plot(navigator='slider')

In [ ]:
hs_stack.swap_axes(1,2).plot(navigator='slider')

In [ ]:
Stack=np.swapaxes(Stack,1,2)

hs_stack_swapped = hs.signals.Signal2D(Stack)
hs_stack_swapped.plot(navigator='slider')

Let's try to reconstruct one slice.

In [ ]:
Stack.shape[0]

In [ ]:
step=2
theta = np.linspace(0., 180., Stack.shape[0], endpoint=False)
sinogram=Stack[::step,75,:]
print(sinogram.shape)
theta= theta[::step]
print(theta)

In [ ]:
print(sinogram.shape)

reconstruction_fbp = iradon(np.swapaxes(sinogram,0,1), theta=theta, circle=True)


theta_rad = (np.pi/180)*theta
vol_geom = astra.create_vol_geom(sinogram.shape[1],sinogram.shape[1])
proj_geom = astra.create_proj_geom('parallel', 1.0, sinogram.shape[1], theta_rad)

# For CPU-based algorithms, a "projector" object specifies the projection
# model used. In this case, we use the "strip" model.
proj_id = astra.create_projector('strip', proj_geom, vol_geom)

# Create a data object for the reconstruction
sinogram_id =astra.data2d.create('-sino',proj_geom,sinogram)
rec_id = astra.data2d.create('-vol', vol_geom)

# Set up the parameters for a reconstruction algorithm using the CPU
# The main difference with the configuration of a GPU algorithm is the
# extra ProjectorId setting.
cfg = astra.astra_dict('SIRT')
cfg['ReconstructionDataId'] = rec_id
cfg['ProjectionDataId'] = sinogram_id
cfg['ProjectorId'] = proj_id
#cfg['options']={'MinConstraint':0}

# Available algorithms:
# ART, SART, SIRT, CGLS, FBP


# Create the algorithm object from the configuration structure
alg_id = astra.algorithm.create(cfg)

# Run 20 iterations of the algorithm
# This will have a runtime in the order of 10 seconds.
astra.algorithm.run(alg_id,20)

# Get the result
rec_astra = astra.data2d.get(rec_id)


# Clean up.
astra.algorithm.delete(alg_id)
astra.data2d.delete(rec_id)
astra.data2d.delete(sinogram_id)
astra.projector.delete(proj_id)



In [ ]:
theta_rad = (np.pi/180)*theta
sinogram_1=Stack[::step,75:76,:]
print(sinogram_1.shape)
print(theta_rad.shape)

recon_tv = tomopy.recon(sinogram_1,theta_rad,algorithm='tv',num_iter=500,reg_par=500)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 2.25))
ax1.set_title("FBP")
ax1.imshow(reconstruction_fbp, cmap=plt.cm.Greys_r)
ax2.imshow(rec_astra, cmap=plt.cm.Greys_r)
ax2.set_title("SIRT")
ax3.imshow(recon_tv[0], cmap=plt.cm.Greys_r)
ax3.set_title("TV")
plt.show()
fig.tight_layout()
plt.show()

In [ ]:
Stack = imread('EELS_1_5_ali.tif')
Stack=np.swapaxes(Stack,1,2)
theta = np.linspace(0,180.,Stack.shape[0])
print(Stack.shape)
print(theta)
plt.figure()
plt.imshow(Stack[0,:,:])

In [ ]:
step=1
theta = np.linspace(0., 180., Stack.shape[0], endpoint=False)
sinogram=Stack[::step,44,:]
print(sinogram.shape)
theta= theta[::step]
print(theta)

In [ ]:
reconstruction_fbp = iradon(np.swapaxes(sinogram,0,1), theta=theta, circle=True)

theta_rad = (np.pi/180)*theta
vol_geom = astra.create_vol_geom(sinogram.shape[1],sinogram.shape[1])
proj_geom = astra.create_proj_geom('parallel', 1.0, sinogram.shape[1], theta_rad)

# For CPU-based algorithms, a "projector" object specifies the projection
# model used. In this case, we use the "strip" model.
proj_id = astra.create_projector('strip', proj_geom, vol_geom)

# Create a data object for the reconstruction
sinogram_id =astra.data2d.create('-sino',proj_geom,sinogram)
rec_id = astra.data2d.create('-vol', vol_geom)

# Set up the parameters for a reconstruction algorithm using the CPU
# The main difference with the configuration of a GPU algorithm is the
# extra ProjectorId setting.
cfg = astra.astra_dict('SIRT')
cfg['ReconstructionDataId'] = rec_id
cfg['ProjectionDataId'] = sinogram_id
cfg['ProjectorId'] = proj_id
#cfg['options']={'MinConstraint':0}

# Available algorithms:
# ART, SART, SIRT, CGLS, FBP


# Create the algorithm object from the configuration structure
alg_id = astra.algorithm.create(cfg)

# Run 20 iterations of the algorithm
# This will have a runtime in the order of 10 seconds.
astra.algorithm.run(alg_id,20)

# Get the result
rec_astra = astra.data2d.get(rec_id)


# Clean up.
astra.algorithm.delete(alg_id)
astra.data2d.delete(rec_id)
astra.data2d.delete(sinogram_id)
astra.projector.delete(proj_id)



In [ ]:
theta_rad = (np.pi/180)*theta
sinogram_1=Stack[::step,44:45,:]
print(sinogram_1.shape)
print(theta_rad.shape)

recon_tv = tomopy.recon(sinogram_1,theta_rad,algorithm='tv',num_iter=200,reg_par=0.0005) #compromise: 200 iter, reg_par 0.0005

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 2.25))
ax1.set_title("FBP")
ax1.imshow(reconstruction_fbp, cmap=plt.cm.Greys_r)
ax2.imshow(rec_astra, cmap=plt.cm.Greys_r)
ax2.set_title("SIRT")
ax3.imshow(recon_tv[0], cmap=plt.cm.Greys_r)
ax3.set_title("TV")
plt.show()
# fig.tight_layout()
# plt.show()

In [ ]:
waveL = imread('EELS_1_5_ali_WT.tif')

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(8, 2.25))
ax1.set_title("FBP")
ax1.imshow(reconstruction_fbp, cmap=plt.cm.Greys_r)
ax2.imshow(rec_astra, cmap=plt.cm.Greys_r)
ax2.set_title("SIRT")
ax3.imshow(recon_tv[0], cmap=plt.cm.Greys_r)
ax3.set_title("TV")
ax4.imshow(waveL[44], cmap=plt.cm.Greys_r)
ax4.set_title("Wavelets")
plt.show()
fig.tight_layout()
plt.show()